In [41]:
import atomium
import math
import mdtraj as md
import nglview
import pybel as bel
import os
from tqdm import tqdm
from rmsd import *
import numpy as np
from pmapper.pharmacophore import Pharmacophore as Pharma
from scipy.spatial.transform import Rotation as Rotate
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc

In [42]:
input_folder = "SI/DockingInput/Ligands_pdb/"
output_folder = "SI/DockingResults/Interface/LigandsOut_0/Ligands_pdb/"

In [43]:
def trans_rot(P,Q):
    
    p = P.mean(axis = 0)
    q = Q.mean(axis = 0)
    T = q-p   #Translation vector
    P += T
    
    R = kabsch(P,Q)   # Rotation matrix
    R = Rotate.from_matrix(R).as_rotvec()
    return np.concatenate([T, np.ndarray.flatten(R)])
    

In [47]:
def get_XY():
    
    X_fingerprints = []
    X_coulomb = []
    X_desc = []
    Y = []
    X_All = []
    
    for file in tqdm(os.listdir(input_folder)):
        try:
            Initial_molecule = Chem.MolFromPDBFile(input_folder+file)
            Final_molecule = Chem.MolFromPDBFile(output_folder+file)
        
            Chem.rdMolTransforms.CanonicalizeMol(Initial_molecule)   # Aligns principal axes with x, y, z
            Initial_coords = Initial_molecule.GetConformers()[0].GetPositions()

            Final_coords = Final_molecule.GetConformers()[0].GetPositions()

            fing = dc.feat.CircularFingerprint()
            x_fingerprint = fing.featurize([Initial_molecule]).flatten()
            X_fingerprints.append(x_fingerprint)

            coulomb_eig = dc.feat.CoulombMatrixEig(max_atoms=200)
            x_coulomb = coulomb_eig._featurize(Initial_molecule).flatten()
            X_coulomb.append(x_coulomb)

            desc = dc.feat.RDKitDescriptors()
            x_desc = desc.featurize([Initial_molecule]).flatten()
            X_desc.append(x_desc)
        
                
        except:
            continue
        
        X_All.append(np.concatenate([x_fingerprint, x_coulomb, x_desc]))
        Y.append(trans_rot(Initial_coords, Final_coords))
        
    return (X_fingerprints, X_coulomb, X_desc, X_All, Y)
    

In [48]:
def main():
    X_f, X_c, X_d, X_All, Y= get_XY()
    print(np.array(X_f))
    print(np.array(X_c))
    print(np.array(X_d))
    print(np.array(X_All))
    print(np.array(Y))

In [ ]:
main()

  1%|          | 108/9121 [00:06<09:23, 16.00it/s]RDKit ERROR: [23:05:09] Explicit valence for atom # 18 C, 5, is greater than permitted
RDKit ERROR: [23:05:09] Explicit valence for atom # 18 C, 5, is greater than permitted
 17%|█▋        | 1510/9121 [01:35<13:09,  9.63it/s]